In [1]:
from collections.abc import Callable

import pandas as pd
import datetime
import os

import FinanceDataReader as fdr
from pandas.core import frame

In [22]:
def get_today() -> tuple[str, str, str]:
  today = datetime.datetime.today()
  year = today.year
  month = today.month
  day = today.day
  return year, month, day

def is_file_in(file_name: str) -> bool:
    year, month, day = get_today()
    csv_file_name = f"{file_name}_{year}_{month}_{day}.csv"
    return os.path.isfile(csv_file_name)

def make_stock_list(file_name: str) -> frame.DataFrame:

    year, month, day = get_today()
    csv_file_name = f"{file_name}_{year}_{month}_{day}.csv"
    if is_file_in(file_name):
        # print(f"File {csv_file_name} exists, skipping.")
        krx_df = pd.read_csv(csv_file_name)
    else:
        # print(f"File {csv_file_name} does not exist, creating it.")
        krx_df = fdr.StockListing("krx")
        # with 구문을 사용해 파일을 명시적으로 열고 닫아준다.
        with open(csv_file_name, "w", newline="", encoding="utf-8") as f:
            krx_df.to_csv(f, index=False)

    return krx_df

stock_list = make_stock_list("krx_df")
stock_list

File krx_df_2024_12_10.csv exists, skipping.


,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,NaN,54000,1,600,1.12,54500,54500,53700,20783970,1122595992277,322368257700000,5969782550,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,NaN,170400,1,1500,0.89,168000,170400,167600,3159112,534512101470,124051602996000,728002365,STK
2,373220,KR7373220003,LG에너지솔루션,KOSPI,NaN,385000,2,-2500,-0.65,392000,396000,384500,249658,96912766500,90090000000000,234000000,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,NaN,959000,1,6000,0.63,962000,966000,946000,86078,82308443000,68255866000000,71174000,STK
4,005380,KR7005380001,현대차,KOSPI,NaN,210500,1,9500,4.73,205500,211500,204000,619392,129273960990,44082108205500,209416191,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2850,245450,KR7245450002,씨앤에스링크,KONEX,일반기업부,1400,3,0,0.00,1400,1400,1400,1,1400,2211944000,1579960,KNX
2851,288490,KR7288490006,나라소프트,KONEX,일반기업부,59,2,-1,-1.67,60,62,51,97539,5172081,1030823043,17471577,KNX
2852,308700,KR7308700004,테크엔,KONEX,일반기업부,199,0,0,0.00,0,0,0,0,0,796000000,4000000,KNX
2853,413300,KR7413300005,티엘엔지니어링,KONEX,일반기업부,550,1,10,1.85,540,550,540,2,1090,718033250,1305515,KNX


In [40]:
today = get_today()
today

(2024, 12, 10)

In [33]:
def get_head(items: list) -> str:
    """
    리스트의 첫 번째 요소를 반환한다.
    리스트가 비어있다면 ValueError를 발생시킨다.
    """
    if not items:
        raise ValueError("The list is empty, cannot get head.")
    return items[0]

def double(fn: Callable) -> Callable:
    return lambda *args: fn(fn(*args))

def find_by_name(name: str) -> frame.DataFrame:
    ser = stock_list["Name"] == name

    stock = stock_list[ser]
    if len(stock) == 0:
        raise ValueError("No stock found")
    return stock

def get_symbol(df: frame.DataFrame) -> str:
    get_double_head = double(get_head)
    if len(df.values) > 0:
        return get_double_head(df.values)
    else:
        raise ValueError("No stock found")

def find_code(name: str) -> str:
    df = find_by_name(name)
    return get_symbol(df)



In [34]:
find_code("AJ네트웍스")

'095570'

In [35]:
stock_code = find_code('CJ제일제당')
print(f"{stock_code=}")
df = fdr.DataReader(symbol=stock_code, start='2024-10')
df

stock_code='097950'


,Open,High,Low,Close,Volume,Change
Date,,,,,,
2024-10-02,301500,303000,290000,292500,76193,-0.037829
2024-10-04,293000,294000,289500,294000,49542,0.005128
2024-10-07,295000,295000,289500,293000,54678,-0.003401
2024-10-08,290500,293000,280500,280500,105208,-0.042662
2024-10-10,283000,290500,279000,287000,80289,0.023173
2024-10-11,287500,287500,280000,280500,65915,-0.022648
2024-10-14,281000,288500,281000,288000,53539,0.026738
2024-10-15,289000,291500,285500,287500,43557,-0.001736
2024-10-16,287000,290000,282500,290000,58266,0.008696
